In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import logging
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
logging.basicConfig(level=logging.INFO)

# Dining Out Scraping

In [4]:
def dining_out_page_out_scraping(zomato_city_link_dining_out):
    # Initialize Chrome WebDriver
    driver = webdriver.Chrome()

    # Load the page
    driver.get(zomato_city_link_dining_out)

    # Scroll down the page to load more content
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Adjust the sleep time as needed
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Get the page source after full rendering
    page_source = driver.page_source

    # Close the WebDriver
    driver.quit()

    # Parse the HTML content
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract restaurant names
    res_names = soup.find_all('h4', class_='sc-1hp8d8a-0 sc-ertOQY hArupt')
    restaurant_names = [res.text for res in res_names]

    # Extract cuisine types
    cuisine_types = soup.find_all('p', class_='sc-1hez2tp-0 sc-iEPtyo bHxcOC')
    restaurant_cuisine_types = [cuisine.text.strip() for cuisine in cuisine_types]

    # Extract pricing
    pricing = soup.find_all('p', class_='sc-1hez2tp-0 sc-iEPtyo dJHUYi')
    restaurant_pricing = [price.text.strip() for price in pricing]

    # Extract locations
    locality = soup.find_all('p', class_='sc-1hez2tp-0 sc-gUlUPW icrStE')
    restaurant_locality = [loc.text.strip() for loc in locality]

    # Extract URLs
    url_tags = soup.find_all('a', class_='sc-hZeNU cijfCJ')
    restaurant_urls = [tag['href'] for tag in url_tags]

    return restaurant_names,restaurant_cuisine_types,restaurant_pricing,restaurant_locality,restaurant_urls



In [5]:
def dining_out_page_in_scraping(restaurant_urls):
    votes = []
    ratings = []
    phone_no = []
    addresses = []
    delivery_ratings = []
    delivery_votes = []
    shop_timing = []
    need_to_drop = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
    }

    for url in restaurant_urls:
        try:
            response = requests.get("https://www.zomato.com"+url, headers=headers)
            response.raise_for_status()  # Raise an exception for non-200 status codes

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract votes
            vote_element = soup.find_all('div', class_='sc-1q7bklc-8 kEgyiI')
            vote = vote_element[0].text.strip() if vote_element else None
            votes.append(vote)

            # Extract delivery votes
            del_vote_element = soup.find_all('div', class_='sc-1q7bklc-8 kEgyiI')
            del_vote = vote_element[1].text.strip() if del_vote_element else None
            delivery_votes.append(del_vote)

            # Extract address
            address_element = soup.find("p", class_="sc-1hez2tp-0 clKRrC")
            address = address_element.text.strip() if address_element else None
            addresses.append(address)

            # Extract phone number
            phone_element = soup.find_all("p", class_="sc-1hez2tp-0 fanwIZ")
            if len(phone_element) <=1:
                phone = phone_element[0].text.strip() if phone_element else None
                phone_no.append(phone)
            else:
                phone = [i.text.strip() for i in phone_element]
                phone_no.append(",".join(phone))

            # Extract ratings
            rating = soup.find_all('div', class_='sc-1q7bklc-1 cILgox')
            first_element_text = rating[0].text.strip() if rating else None
            ratings.append(first_element_text)

            # Extract delivery ratings
            del_rating = soup.find_all('div', class_='sc-1q7bklc-1 cILgox')
            second_element_text = del_rating[1].text.strip() if del_rating else None
            delivery_ratings.append(second_element_text)

            # Extract shop_timing
            timing = soup.find('span', class_='sc-kasBVs dfwCXs')
            if timing is not None:
                shop_timing.append(timing.text.replace(" (Today)", ""))
            else:
                shop_timing.append(None)

        except requests.HTTPError as e:
            print(f"HTTP Error occurred for URL: {url}")
            need_to_drop.append("https://www.zomato.com"+url)

            # Handle the error as needed, such as logging it or skipping the URL
            pass

    return votes, ratings, phone_no, addresses, delivery_ratings, delivery_votes, shop_timing,need_to_drop


In [6]:
def dining_out_restaurants_to_df(zomato_city_link_dining_out):

   # Call functions to scrape data
    restaurant_names,restaurant_cuisine_types,restaurant_pricing,restaurant_locality,restaurant_urls = dining_out_page_out_scraping(zomato_city_link_dining_out)
    votes,ratings ,phone_no ,addresses ,delivery_ratings,delivery_votes,shop_timing,need_to_drop  = dining_out_page_in_scraping(restaurant_urls)

    # Extract city name from the URL
    city_name = zomato_city_link_dining_out.split("/")[-1].capitalize()
    restaurant_full_urls = ["https://www.zomato.com" + url for url in restaurant_urls]
    # Create DataFrame
    df = pd.DataFrame({
    "Restaurant_Name": restaurant_names,
    "Locality": restaurant_locality,
    "Cuisines": restaurant_cuisine_types,
    "Average_Cost_for_two": restaurant_pricing,
    "Restaurant_Url":restaurant_full_urls})
    df = df[~df.Restaurant_Url.isin(need_to_drop)]
    df = df.reset_index(drop= True)
   
    df1 = pd.DataFrame({ 
        "City": city_name,
        "Address": addresses,
        "Available_Time": shop_timing,
        "Contact_Number": phone_no,
        "Dining_Ratings": ratings,
        "Dining_Votes": votes,
        "Delivery_Ratings": delivery_ratings,
        "Delivery_Votes": delivery_votes
    })
    df2 = pd.merge(df,df1,how = "left",left_index=True, right_index=True)

    return df2

In [7]:
gwalior_df = dining_out_restaurants_to_df("https://www.zomato.com/gwalior")

In [12]:
gwalior_df.to_csv(r"D:\DATA SCIENCE ZONE\Projects\guvi_final_project\Zomato-Data-Analysis-and-Visualization\datasets\city_wise_restaurants_data_set\gwalior_df.csv", index = False)

In [8]:
gwalior_df

,Restaurant_Name,Locality,Cuisines,Average_Cost_for_two,Restaurant_Url,City,Address,Available_Time,Contact_Number,Dining_Ratings,Dining_Votes,Delivery_Ratings,Delivery_Votes
0,Burger King,"DB City Mall, Birla Nagar, Gwalior","Burger, Fast Food, Beverages, Desserts",₹400 for two,https://www.zomato.com/gwalior/burger-king-bir...,Gwalior,"Unit T 6 &7, DB Mall, Old MPRTC, Workshop Race...","12midnight – 3am, 10am – 12midnight",+918657446793,4.0,689,3.9,33.1K
1,Sagar Gaire Fast Food,"City Center, Gwalior","Fast Food, North Indian, South Indian, Chinese",₹350 for two,https://www.zomato.com/gwalior/sagar-gaire-fas...,Gwalior,"G-193, Bds Tower, Near Green Garden Patel Naga...",Opens at 11am,+917415676726,New,1,4.0,11
2,ChotiWala Restaurant,"City Center, Gwalior","North Indian, Chinese",₹350 for two,https://www.zomato.com/gwalior/chotiwala-resta...,Gwalior,"Near Station Road, Station Bajariya, City Cent...",9am – 11:45pm,+917610720020,4.0,413,4.0,"9,594"
3,Velkim - Welcome Bakery,"Lashkar, Gwalior","Bakery, South Indian, Chinese, North Indian, F...",₹500 for two,https://www.zomato.com/gwalior/velkim-welcome-...,Gwalior,"Opposite Old High Court, Lashkar, Gwalior",Opens at 11am,+919893272368,-,1,4.3,66
4,Rolls Mania,"City Center, Gwalior","Rolls, North Indian, Fast Food, Street Food, B...",₹400 for two,https://www.zomato.com/gwalior/rolls-mania-cit...,Gwalior,"J 812, A Shyamla Building, Gokulvihar Chouraha...",Opens at 11:30am,+918826160308,3.7,31,4.3,"1,347"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,Hotel Raj Mohan Palace,"Anand Nagar, Gwalior","North Indian, Chinese, Beverages",₹600 for two,https://www.zomato.com/gwalior/hotel-raj-mohan...,Gwalior,"National Highway 3, Anand Nagar, Gwalior",11am – 11:30pm,+917089003315,-,0,-,0
488,Shri Ram Bhojnalay And Restaurant,"Deen Dayal Nagar, Gwalior",North Indian,₹350 for two,https://www.zomato.com/gwalior/shri-ram-bhojna...,Gwalior,"Opposite BSF Colony, Bhind Road, Maharajpura, ...",11am – 10pm,+919300195059,-,1,-,1
489,Rajhans Hotel And Restaurant,"Lashkar, Gwalior",North Indian,₹300 for two,https://www.zomato.com/gwalior/rajhans-hotel-a...,Gwalior,"Sikroda Circle, Near Malwa College, Jhansi Roa...",10am – 12midnight,+918463023183,-,0,-,0
490,Hotel Highway Glory,"Thatipur, Gwalior","South Indian, North Indian, Chinese, Desserts,...",₹800 for two,https://www.zomato.com/gwalior/hotel-highway-g...,Gwalior,"105 KM, Milestone Agra, Jhansi Highway, Thatip...",7am – 11pm,+919977600912,-,0,-,0
